In [0]:
%%bash
pip install tsfresh
pip install ipywidgets
jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [0]:
import os
import inspect
import sys
import pandas as pd
import charts
import numpy as np
import pylab as pl
from scipy import spatial
from scipy.spatial import distance
from sklearn.cluster import MeanShift
import time
import math
from opengrid.library import houseprint

In [0]:
hp = houseprint.Houseprint()

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

In [0]:
gas = hp.find_sensor('212ce724e124fbde0fb649396375d099')
#gas = hp.find_sensor('a6028457292f1866b468b63adcc82ee3') 2
#gas = hp.find_sensor('a6028457292f1866b468b63adcc82ee3')

In [0]:
head = pd.Timestamp('20151110')
tail = pd.Timestamp('20151128')
originalGraph = gas.get_data(head=head, tail=tail, diff=True, resample='min', unit='kW')
originalGraphDf=pd.DataFrame(originalGraph)
originalGraphDfDropNa = originalGraphDf.dropna()
charts.plot(originalGraph, stock=True, show='inline')

In [0]:
dfCompareOriginal=originalGraphDf.ix[pd.Timestamp('2015-11-13 16:25:00+00:00'):pd.Timestamp('2015-11-13 17:16:00+00:00')]

<h3> Different compare methods </h3>

<h4> Manhattan Distance: the method we will use </h4>

In [0]:
def manhattanDistance2(s1,s2):
    dist = distance.cityblock(s1,s2)
#     maxValues=np.maximum(s1,s2)
#     maxValues=np.max(maxValues)
#     manhattan=dist/maxValues
    return dist

# Storing comparing arrays used in the methods below

In [0]:
dfCompareOriginal=originalGraphDf.ix[pd.Timestamp('2015-11-13 16:29:00+00:00'):pd.Timestamp('2015-11-13 17:14:00+00:00')]

# Testing Graph Method

<h3> Making methods more clear </h3>

In [0]:
 def looping_through_list(df_with_values, array_to_compare):
    start=time.time()
    #Create list where values on which the different methods will act will be saved temporarily
    #This list will be cleared every time
    values_list=[]
    #Create list with results of the methods
    result_list=[]
    compare_to_array=np.array(array_to_compare)
    i=0 #Loops through list arrayToCompare.size times
    j=0 #Amount of values considered, and calculated.
    array_with_values=np.array(df_with_values)
    #start and stop timestamp
    starting_time_stamp=df_with_values.index[0]
    stop_time_stamp=df_with_values.index[array_to_compare.size-1]
    
    #Threshold
    threshold_value=array_to_compare.size/10
    #LOOPING THROUGH LIST
    while i < array_to_compare.size+j: #i is dependent on j, i updates as j updates.
        #Save x amount of values where x is the size of the array to compare.
        values_list.append(([array_with_values[i][0]]))
        i=i+1
        if(i==(array_to_compare.size+j)) and (i<(df_with_values.size)):
            #j = the minute at this moment. i = starting from the current minute, adding the size of the array to compare.
            #Stops running when the limit of values to be considered is reached, being the size the original array.
            if(j==0) or (manhattanDistance2(compare_to_array,values_list)!=result_list[len(result_list)-1][0]):
                #Create list with unique values. Save them in "resultList". Euclidean score, startTimestamp, stopTimestamp
                result_list.append([manhattanDistance2(compare_to_array,values_list), starting_time_stamp,stop_time_stamp]) 
            j=j+1
            starting_time_stamp=df_with_values.index[j] #startingTimeStamp: current minute considered
            stop_time_stamp=df_with_values.index[i] #stopTimeStamp: current minute considered + size of array to compare.
            i=j
            values_list=[]
    print time.time()-start
    return result_list

In [0]:
 def local_minima_ts_correction(result_list):
    start=time.time()
    loop_index=0 #Loops throu"gh the resultslist, as long as its size
    current_saves=0 #Current amount of variables saved
    some_list=[]
    df_result=pd.DataFrame(some_list)
    ts=(list_view[0][2]-list_view[0][1])/10
    threshold_value=(ts / np.timedelta64(1, 'm')).astype(int)
    #SAVING ONLY RELEVANT VALUES       
    while loop_index < len(result_list):
        if(loop_index==0):
            [value,start_time_stamp,stop_time_stamp] = [result_list[loop_index][0], result_list[loop_index][1], result_list[loop_index][2]]
            df_result=df_result.append(pd.DataFrame([[value, start_time_stamp,stop_time_stamp]], index=[current_saves], columns=['Manhattan','startTimeStamp','stopTimeStamp']))
            current_saves=current_saves+1
        if(result_list[loop_index-1][0] < result_list[loop_index-2][0]) and (result_list[loop_index-1][0] < result_list[loop_index][0]) and result_list[loop_index-1][0] < df_result.max()['Manhattan']:
            #print "Value", resultList[loopIndex-1][0],",Index:",loopIndex-1,"has a lower value than value left and right to it.(",resultList[loopIndex-2][0],",",resultList[loopIndex][0],") nIt will now replace", dfResult['Euclidean'].max(),"in the dataset."
            [value,start_time_stamp,stop_time_stamp] = [result_list[loop_index-1][0], result_list[loop_index-1][1], result_list[loop_index-1][2]]
            list_with_values=[]
            i=0
            max_value=0
            #!!!Timestamps are important. If timestamp is within the range of another timestamp already present,
            #they will overwrite eachother instead of adding a new unique value
            while i < df_result.index.size:
                if(start_time_stamp >= df_result['startTimeStamp'][i]) and (start_time_stamp <= df_result['stopTimeStamp'][i]-pd.Timedelta(minutes=threshold_value)):
                    #INSIDE BOUNDARIES
                    if(df_result.loc[i][0] > max_value) and (df_result.loc[i][0]!=0): #Store the maximum value, of the range between start and stoptimestamp.
                        max_value=df_result.loc[i][0]
                        ts=df_result.loc[i][1]
                    elif(df_result.loc[i][0]==0): #PERFECT MATCH
                        max_value=1 #Not 0 because otherwise we assume it is outside of the boundaries
                i=i+1
            if(max_value==0): 
                #OUTSIDE BOUNDARIES
                    df_result=df_result.append(pd.DataFrame([[value, start_time_stamp,stop_time_stamp]], index=[current_saves], columns=['Manhattan','startTimeStamp','stopTimeStamp']))
                    current_saves=current_saves+1
                #INSIDE BOUNDARIES - 
            elif(value < max_value): #Check if the current value is smaller than the Maximum value encountered. Replace if it is.
                    df_result.loc[df_result['Manhattan']== max_value] = [value, start_time_stamp, stop_time_stamp]
        loop_index=loop_index+1
    df_result=df_result.sort_values(['Manhattan'])
    print time.time()-start
    return df_result
        

In [0]:
def calculate_threshold_value(df_result, nr_of_best_values, factor):
    list_of_best_values=[]
    i = 0
    for i in range(0, nr_of_best_values):
        list_of_best_values.append(df_result.iloc[i])
    list_of_best_values=pd.DataFrame(list_of_best_values)
    mean=list_of_best_values.mean()
    threshold=mean*factor
    print threshold
    return df_result.loc[df_result['Manhattan']<=threshold[0]]
    

In [0]:
list_view=looping_through_list(originalGraphDfDropNa, dfCompareOriginal)
df_result = local_minima_ts_correction(list_view)

In [0]:
list_threshold=calculate_threshold_value(df_result, 20, 2)

In [0]:
#first_half_df = pd.DataFrame(columns=['first_half_median'])
#second_half_df = pd.DataFrame(columns=['second_half_median'])
#for i in range(0, list_threshold['Manhattan'].size):
#    ts_half = list_threshold['startTimeStamp'].iloc[i] + (list_threshold['stopTimeStamp'].iloc[i] - list_threshold['startTimeStamp'].iloc[i])/2
#    first_median = originalGraphDfDropNa.ix[list_threshold['startTimeStamp'].iloc[i]: ts_half].mean().iloc[0]
#    second_median = originalGraphDfDropNa.ix[ts_half : list_threshold['stopTimeStamp'].iloc[i]].mean().iloc[0]
#    index = list_threshold.index[i]
#    
#    first_half_df = first_half_df.append(pd.DataFrame(data=[first_median], index=[index], columns=['first_half_median']))
#    second_half_df = second_half_df.append(pd.DataFrame(data=[second_median], index=[index], columns=['second_half_median']))
#
#split_result = pd.concat([list_threshold,first_half_df,second_half_df],axis=1).sort_values('startTimeStamp')

In [0]:
def split(original, event_list, parts):
    
    cols = []
    for i in range(0, parts):
        cols.extend(['part_' + str(i)])    
    parts_df = pd.DataFrame(columns=cols, index=list_threshold.index)
    
    for i in range(0, event_list['Manhattan'].size):
        part_length = (event_list['stopTimeStamp'].iloc[i] - event_list['startTimeStamp'].iloc[i])/parts
        
        prev_end_time = event_list['startTimeStamp'].iloc[i]
        for j in range(0, parts):
            part_median = original.ix[prev_end_time: prev_end_time + part_length].mean().iloc[0]
            prev_end_time = prev_end_time + part_length
            index = list_threshold.index[i]
            
            parts_df.set_value(index, 'part_' + str(j), part_median)
            
    split_list = pd.concat([event_list, parts_df],axis=1)
    
    return split_list

In [0]:
test = split(originalGraphDfDropNa, list_threshold, 2)

In [0]:
def create_graph_from(df):
    listWithGraphs=[]
    listWithGraphs=pd.DataFrame(listWithGraphs)
    for i in range (0, df.index.size):
        listWithGraphs=listWithGraphs.append(originalGraphDfDropNa.ix[pd.Timestamp(df['startTimeStamp'][i]):pd.Timestamp(df['stopTimeStamp'][i])])
    return charts.plot(listWithGraphs, stock=True, show='inline')

In [0]:
df_result = pd.DataFrame(data=[(test['Manhattan']/test['Manhattan'].max()).as_matrix(),(test['part_0']/test['part_0'].max()).as_matrix(),(test['part_1']/test['part_1'].max()).as_matrix()]).transpose()
result=df_result.as_matrix()

In [0]:
ms = MeanShift()
ms.fit(result)
labels = ms.labels_
cluster_centers = ms.cluster_centers_
n_cluster = len(np.unique(labels))
colors = 10*['g.', 'r.', 'c.', 'b.', 'k.', 'y.', 'm.']

In [0]:
try:
    del test['Type']
except Exception as e:
    print 'No column named Type'
    
    
test.insert(loc=5, column='Type', value=labels)

In [0]:
test.sort_values('Manhattan')

In [0]:
print labels
for i in range(0, test.index.size):
    if test['Type'].iloc[i] == 4 or test['Type'].iloc[i] == 5:
        plt.plot(originalGraphDfDropNa.ix[pd.Timestamp(test['startTimeStamp'].iloc[i]) - pd.Timedelta(minutes=2):pd.Timestamp(test['stopTimeStamp'].iloc[i]) + pd.Timedelta(minutes=2)])     

In [0]:
print labels
for i in range(0, 13):
    #if test['Type'].iloc[i] == 0 or test['Type'].iloc[i] == 0:
    plt.plot(originalGraphDfDropNa.ix[pd.Timestamp(test['startTimeStamp'].iloc[i]):pd.Timestamp(test['stopTimeStamp'].iloc[i])])     

In [0]:
charts.plot(dfCompareOriginal, show='inline', stock=True)